In [31]:
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dimod import BinaryQuadraticModel
import numpy as np

In [32]:
bqm=BinaryQuadraticModel('BINARY')

### Create N-Queen problem matrix $Q$

In [73]:
N=15

In [74]:
class QuboPoly():
    def __init__(self, n=1024):
        self.array=np.zeros((n, n), dtype=int)
        self.constant=0
        self._size=n
    
    def add_term(self, i, j, c):
        if i>=self._size or j>=self._size:
            raise RuntimeError("Wrong index")
        self.array[i][j]+=c
        
    def add_constant(self, c):
        self.constant+=c
        
    def sum(self, p):
        if self._size != p._size:
            raise RuntimeError("Wrong polynomial size")
        self.array+=p.array
        self.constant+=p.constant
        
    def power(self):
        a=np.diag(self.array)
        self.array=np.outer(a, a) + 2*self.constant*np.diag(a) # convert back to NxN
        self.constant**=2
        
    def multiply(self, p):
        a=np.diag(self.array)
        b=np.diag(p.array)
        self.array=np.outer(a, b) + self.constant*np.diag(b) + p.constant*np.diag(a)
        self.constant*=p.constant
        
def build_column_penalty(N):
    qubo=QuboPoly(N*N)
    for col in range(N):
        tmp=QuboPoly(N*N)
        for i in grid[:, col]:
            tmp.add_term(i, i, 1)
        tmp.add_constant(-1)
        tmp.power()
        qubo.sum(tmp)
    return qubo

def build_row_penalty(N):
    qubo=QuboPoly(N*N)
    for row in range(N):
        tmp=QuboPoly(N*N)
        for i in grid[row]:
            tmp.add_term(i, i, 1)
        tmp.add_constant(-1)
        tmp.power()
        qubo.sum(tmp)
    return qubo

def build_diag_penalty(N):
    qubo=QuboPoly(N*N)
    for g in [grid, np.fliplr(grid)]:
        for k in range(-N+2, N-1):
            tmp1=QuboPoly(N*N)
            tmp2=QuboPoly(N*N)
            for dia in np.diag(g, k=k):
                tmp1.add_term(dia, dia, 1)
                tmp2.add_term(dia, dia, 1)
            tmp1.add_constant(-1)
            tmp1.multiply(tmp2)
            qubo.sum(tmp1)
    return qubo

grid=np.arange(N*N).reshape(N, N)
P1=build_column_penalty(N)
P2=build_row_penalty(N)
P3=build_diag_penalty(N)
qubo_obj=QuboPoly(N*N)
qubo_obj.sum(P1)
qubo_obj.sum(P2)
qubo_obj.sum(P3)
print(qubo_obj.array)
print(qubo_obj.constant)

[[-2  1  1 ...  0  0  1]
 [ 1 -2  1 ...  0  0  0]
 [ 1  1 -2 ...  0  0  0]
 ...
 [ 0  0  0 ... -2  1  1]
 [ 0  0  0 ...  1 -2  1]
 [ 1  0  0 ...  1  1 -2]]
30


In [75]:
def example():
    qubo=np.array([[-5,2,4,0],
                   [2,-3,1,0],
                   [4,1,-8,5],
                   [0,0,5,-6]])
    return qubo

In [76]:
from collections import defaultdict
Q=defaultdict(int)

for i in range(qubo_obj._size):
    for j in range(qubo_obj._size):
        Q[(i, j)]=qubo_obj.array[i][j]

#Q=example()
print(len(Q))

50625


In [77]:
bqm=BinaryQuadraticModel.from_qubo(Q)

### Using Leap’s Hybrid Solvers

In [60]:
from time import monotonic
start_time=monotonic()  # timer

bqm_sampler=LeapHybridSampler()
sampleset=bqm_sampler.sample(bqm, label='N-Queen', time_limit=5)

print(f"Run time {monotonic() - start_time} seconds")

Run time 12.048413621000236 seconds


In [61]:
print(sampleset.info)

{'qpu_access_time': 145702, 'charge_time': 4993578, 'run_time': 4993578, 'problem_id': '31a3b9ce-6c21-49fc-ac35-d116707df23d', 'problem_label': 'N-Queen'}


In [62]:
best_sample=sampleset.first.sample
best_energy=sampleset.first.energy

print("Best Sample:", best_sample)
print("Best Energy:", best_energy)

# map it back to answer
solution=np.zeros(N*N, dtype=int)
for index, var in enumerate(best_sample):
    solution[index]=int(best_sample[var])
    
board_sol=solution.reshape(N, N)
print(board_sol)

Best Sample: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 1, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 1, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 1, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 1, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 1, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 1, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 

### Use D-Wave's 2000Q solver

In [104]:
start_time=monotonic()  # timer

sampler=EmbeddingComposite(DWaveSampler())
sampleset=sampler.sample(bqm, num_reads=5)

print(f"Run time {monotonic() - start_time} seconds")

Run time 67.65822556500007 seconds


In [105]:
for s, e in sampleset.data(['sample', 'energy']):
    print("Sample:", s, "Energy:", e)
    
    '''
    solution=np.zeros(N*N, dtype=int)
    for index, var in enumerate(s):
        solution[index]=int(s[var])
    board_sol=solution.reshape(N, N)
    ob=Solution()
    if ob.solve(board_sol)==True:
        print("YES")
    '''
    
#print("Skip")

Sample: {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 0, 41: 0, 42: 1, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 1, 53: 0, 54: 0, 55: 0, 56: 0, 57: 1, 58: 0, 59: 0, 60: 0, 61: 0, 62: 1, 63: 0} Energy: -8.0
Sample: {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 1, 37: 0, 38: 0, 39: 1, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 1, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 1, 57: 0, 58: 0, 59: 0, 60: 0, 61: 1, 62: 0, 63: 0} Energy: -4.0
Sample: {0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 

In [106]:
first_sample=sampleset.first.sample
first_energy=sampleset.first.energy

#print("First Sample:", first_sample)
print("Energy of First Sample:", first_energy)

Energy of First Sample: -8.0


**Quantum annealing** is a stochastic process, and the samples generated represent possible solutions to the problem based on the quantum fluctuations and thermal noise inherent in the hardware.

In [107]:
best_energy=float('inf')
best_sample=None

for s, e in sampleset.data(['sample', 'energy']):
    if e < best_energy:
        best_energy=e
        best_sample=s

#print("Best Sample:", best_sample)
print("Best Energy:", best_energy)

Best Energy: -8.0


In [108]:
# map it back to answer
solution=np.zeros(N*N, dtype=int)
for index, var in enumerate(best_sample):
    solution[index]=int(best_sample[var])
    
board_sol=solution.reshape(N, N)
print(board_sol)

[[1 0 0 0 0 1 0 0]
 [0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 1 0]]


#### Validate Solution

In [63]:
class Solution:
    def solve(self, matrix):
        n=len(matrix)
        rows=set()
        cols=set()
        diags=set()
        rev_diags=set()

        for i in range(n):
            for j in range(n):
                if matrix[i][j]:
                    rows.add(i)
                    cols.add(j)
                    diags.add(i-j)
                    rev_diags.add(i+j)

        return len(rows)==len(cols)==len(diags)==len(rev_diags)==n

ob=Solution()
print("Answer =", ob.solve(board_sol))

Answer = True
